In [6]:
from glob import glob as glob
from pathlib import Path
from sklearn import preprocessing

csvs = glob('./BayesNetOutputs/Absolute/*_Matrix.csv')

In [10]:
csvs

['./BayesNetOutputs/Absolute/VIC_LD_Matrix.csv',
 './BayesNetOutputs/Absolute/VIC_Matrix.csv',
 './BayesNetOutputs/Absolute/TAS_Matrix.csv',
 './BayesNetOutputs/Absolute/NSW_SWS_Matrix.csv',
 './BayesNetOutputs/Absolute/ACT_Matrix.csv',
 './BayesNetOutputs/Absolute/BM_Matrix.csv',
 './BayesNetOutputs/Absolute/NSW_SEC_Matrix.csv',
 './BayesNetOutputs/Absolute/QLD_Matrix.csv',
 './BayesNetOutputs/Absolute/SA_Matrix.csv',
 './BayesNetOutputs/Absolute/NSW_NAN_Matrix.csv',
 './BayesNetOutputs/Absolute/NSW_BHILL_Matrix.csv']

name

In [73]:
for i in csvs:
    df = pd.read_csv(i)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    name = df['region'][0]
    cols = df.columns.to_list()
    cols.remove('Edge_Treatment')
    cols.remove('Landscape_Treatment')
    cols.remove('region')
    
    x = df[cols].values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, columns=cols)
    fname = 'BayesNetOutputs/'+ name + '_normalised.csv'
    df.to_csv(fname)
    print('Wrote: %s' % fname)

Wrote: BayesNetOutputs/VIC_LD_normalised.csv
Wrote: BayesNetOutputs/VIC_normalised.csv
Wrote: BayesNetOutputs/TAS_normalised.csv
Wrote: BayesNetOutputs/NSW_SWS_normalised.csv
Wrote: BayesNetOutputs/ACT_normalised.csv
Wrote: BayesNetOutputs/BM_normalised.csv
Wrote: BayesNetOutputs/NSW_SEC_normalised.csv
Wrote: BayesNetOutputs/QLD_normalised.csv
Wrote: BayesNetOutputs/SA_normalised.csv
Wrote: BayesNetOutputs/NSW_NAN_normalised.csv
Wrote: BayesNetOutputs/NSW_BHILL_normalised.csv
